# HelixGraph - Neo4j Cypher Queries

This notebook contains useful Cypher queries to explore and analyze the HelixGraph database.

## Setup - Connect to Neo4j

In [153]:
# Set up project path
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from dotenv import load_dotenv
from neo4j import GraphDatabase
import pandas as pd

# Load environment variables
load_dotenv(os.path.join(project_root, '.env'))

# Neo4j connection
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USER = os.getenv('NEO4J_USER')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def run_query(query, params=None):
    """Execute a Cypher query and return results as a DataFrame"""
    with driver.session() as session:
        result = session.run(query, params or {})
        return pd.DataFrame([record.data() for record in result])

print("✓ Connected to Neo4j")
print(f"  URI: {NEO4J_URI}")

✓ Connected to Neo4j
  URI: neo4j+s://561f8654.databases.neo4j.io


## 1. Database Overview

### Total Node and Relationship Counts

In [154]:
query = """
MATCH (n) 
RETURN count(n) AS total_nodes;
"""

result = run_query(query)
print(f"Total Nodes: {result['total_nodes'][0]:,}")
result

Total Nodes: 9,793


,total_nodes
0,9793


In [155]:
query = """
MATCH ()-[r]->() 
RETURN count(r) AS total_relationships;
"""

result = run_query(query)
print(f"Total Relationships: {result['total_relationships'][0]:,}")
result

Total Relationships: 27,131


,total_relationships
0,27131


### Node Count by Label

In [156]:
query = """
MATCH (n)
UNWIND labels(n) AS label
RETURN label, count(*) AS node_count
ORDER BY node_count DESC;
"""

result = run_query(query)
print("Node counts by label:")
result

Node counts by label:


,label,node_count
0,PurchaseOrderLine,4331
1,CommerceOrder,1828
2,PurchaseOrder,1747
3,MarketingCampaign,600
4,SupplierRisk,534
5,Contract,313
6,Employee,200
7,Supplier,150
8,Skill,50
9,Brand,10


### Relationship Count by Type

In [157]:
query = """
MATCH ()-[r]->()
RETURN type(r) AS relationship_type, count(r) AS rel_count
ORDER BY rel_count DESC;
"""

result = run_query(query)
print("Relationship counts by type:")
result

Relationship counts by type:


,relationship_type,rel_count
0,KPI_RESULT,8965
1,HAS_LINE,4331
2,FULFILLED_BY,4331
3,ATTRIBUTED_TO,1828
4,ACTIVATED_ON,1793
5,PLACED_ORDER,1747
6,HAS_SKILL,1496
7,FOR_BRAND,600
8,HAS_OBJECTIVE,600
9,HAS_RISK,534


### All Node Labels

In [158]:
query = """
CALL db.labels();
"""

result = run_query(query)
print("All node labels in the database:")
result

All node labels in the database:


,label
0,Employee
1,Department
2,Location
3,Skill
4,MarketingCampaign
5,Brand
6,MarketingObjective
7,MarketingChannel
8,MarketingKPI
9,CommerceOrder


### All Relationship Types

In [159]:
query = """
CALL db.relationshipTypes();
"""

result = run_query(query)
print("All relationship types in the database:")
result

All relationship types in the database:


,relationshipType
0,WORKS_IN
1,LOCATED_IN
2,REPORTS_TO
3,HAS_SKILL
4,SUB_CHANNEL_OF
5,FOR_BRAND
6,HAS_OBJECTIVE
7,ACTIVATED_ON
8,KPI_RESULT
9,ATTRIBUTED_TO


## 2. HR Domain Queries

### Employees by Department

In [160]:
query = """
MATCH (d:Department)<-[:WORKS_IN]-(e:Employee) 
RETURN d.name AS dept, count(DISTINCT e) AS employees 
ORDER BY employees DESC;
"""

result = run_query(query)
print("Employees by department:")
result

Employees by department:


,dept,employees
0,Finance,39
1,Engineering,38
2,Operations,33
3,HR,32
4,Marketing,30
5,Sales,28


### All Departments

In [161]:
query = """
MATCH (d:Department) 
RETURN d.name AS dept_name, count(*) AS cnt 
ORDER BY dept_name;
"""

result = run_query(query)
print("All departments:")
result

All departments:


,dept_name,cnt
0,Engineering,1
1,Finance,1
2,HR,1
3,Marketing,1
4,Operations,1
5,Sales,1


### Most Common Skills Across All Departments

In [162]:
query = """
MATCH (d:Department)<-[:WORKS_IN]-(e:Employee)-[:HAS_SKILL]->(s:Skill) 
WHERE d.name IN ["HR", "Marketing", "Procurement"] 
RETURN s.name AS skill_name, s.category AS category, count(DISTINCT e) AS employee_count 
ORDER BY employee_count DESC 
LIMIT 20;
"""

result = run_query(query)
print("Most common skills across HR, Marketing, and Procurement:")
result

Most common skills across HR, Marketing, and Procurement:


,skill_name,category,employee_count
0,Time Management,Soft Skill,50
1,English,Language,49
2,Communication,Soft Skill,48
3,Excel,Tool,47
4,Project Management,Soft Skill,35
5,SEO/SEM,Domain,29
6,Content Strategy,Domain,28
7,Recruitment,Domain,26
8,Digital Marketing,Domain,24
9,HR Policies,Domain,22


### Top 10 Employees by Skill Count

In [163]:
query = """
MATCH (e:Employee)-[:HAS_SKILL]->(s:Skill)
WITH e, count(s) AS skill_count
RETURN e.first_name + ' ' + e.last_name AS employee_name, 
       e.job_title AS job_title,
       skill_count
ORDER BY skill_count DESC
LIMIT 10;
"""

result = run_query(query)
print("Top 10 employees by number of skills:")
result

Top 10 employees by number of skills:


,employee_name,job_title,skill_count
0,Rodney Bell,Finance Manager,12
1,Craig Faulkner,Software Engineer,12
2,Nicole Tucker,Engineering Manager,12
3,Kathleen Kim,Senior Engineer,12
4,Katelyn Stewart,Finance Manager,12
5,Melvin Alvarado,VP of Sales,12
6,Cameron West,Senior Engineer,12
7,Jim Underwood,Engineering Manager,12
8,Stephanie Wolf,Tech Lead,11
9,Chad Collins,Engineering Manager,11


## 3. Marketing Domain Queries

### Campaign Performance Overview

In [164]:
query = """
MATCH (c:MarketingCampaign)
OPTIONAL MATCH (o:CommerceOrder)-[:ATTRIBUTED_TO]->(c)
WITH c, count(o) AS order_count, sum(o.total_value) AS total_revenue
RETURN c.name AS campaign_name,
       c.budget AS budget,
       order_count,
       coalesce(total_revenue, 0) AS total_revenue,
       c.start_date AS start_date,
       c.end_date AS end_date
ORDER BY total_revenue DESC
LIMIT 10;
"""

result = run_query(query)
print("Top 10 campaigns by revenue:")
result

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: total_value)} {position: line: 4, column: 40, offset: 123} for query: '\nMATCH (c:MarketingCampaign)\nOPTIONAL MATCH (o:CommerceOrder)-[:ATTRIBUTED_TO]->(c)\nWITH c, count(o) AS order_count, sum(o.total_value) AS total_revenue\nRETURN c.name AS campaign_name,\n       c.budget AS budget,\n       order_count,\n       coalesce(total_revenue, 0) AS total_revenue,\n       c.start_date AS start_date,\n       c.end_date AS end_date\nORDER BY total_revenue DESC\nLIMIT 10;\n'


Top 10 campaigns by revenue:


,campaign_name,budget,order_count,total_revenue,start_date,end_date
0,User-centric even-keeled encryption,179471.0,4,0,2025-01-21,2025-03-21
1,Face-to-face client-driven support,143143.0,2,0,2025-01-31,2025-03-05
2,Customer-focused systematic support,225691.0,2,0,2025-03-31,2025-06-15
3,Quality-focused background parallelism,176787.0,4,0,2025-01-05,2025-03-29
4,Optimized 5thgeneration algorithm,186450.0,2,0,2025-03-14,2025-05-02
5,Cross-group didactic emulation,188992.0,3,0,2025-03-11,2025-05-09
6,Profit-focused real-time algorithm,93653.0,4,0,2025-04-06,2025-06-09
7,Realigned dedicated structure,168184.0,2,0,2025-03-27,2025-05-20
8,Streamlined tangible moratorium,130886.0,4,0,2025-02-06,2025-03-14
9,Sharable bifurcated algorithm,137314.0,3,0,2025-01-04,2025-02-08


### Marketing Channels Usage

In [165]:
query = """
MATCH (ch:MarketingChannel)<-[:ACTIVATED_ON]-(c:MarketingCampaign)
RETURN ch.name AS channel_name,
       ch.type AS channel_type,
       count(DISTINCT c) AS campaign_count
ORDER BY campaign_count DESC;
"""

result = run_query(query)
print("Marketing channels by campaign usage:")
result

Marketing channels by campaign usage:


,channel_name,channel_type,campaign_count
0,Paid Search,Search,270
1,Online Video,Display,269
2,Programmatic Display,Display,257
3,Retail,Retail,253
4,Public Relations,PR,249
5,Email,Email,248
6,Social,Social,247


### Brand Campaign Distribution

In [166]:
query = """
MATCH (b:Brand)<-[:FOR_BRAND]-(c:MarketingCampaign)
RETURN b.name AS brand_name,
       count(c) AS campaign_count,
       sum(c.budget) AS total_budget
ORDER BY campaign_count DESC;
"""

result = run_query(query)
print("Campaigns and budgets by brand:")
result

Campaigns and budgets by brand:


,brand_name,campaign_count,total_budget
0,Fjord Living,68,11289567.0
1,VelvetLane,67,11560014.0
2,Luma Cosmetics,63,10490666.0
3,Atlas Groom,63,9856966.0
4,Aurora Hair,61,9573802.0
5,PulseSkin,58,9520829.0
6,VitaCore,57,9197230.0
7,DermaLab,56,9653936.0
8,Noir Essence,54,8824554.0
9,Pure Botanics,53,9153691.0


### Campaign Objectives

In [167]:
query = """
MATCH (o:MarketingObjective)<-[:HAS_OBJECTIVE]-(c:MarketingCampaign)
RETURN o.name AS objective,
       count(c) AS campaign_count,
       sum(c.budget) AS total_budget
ORDER BY campaign_count DESC;
"""

result = run_query(query)
print("Campaigns by objective:")
result

Campaigns by objective:


,objective,campaign_count,total_budget
0,Retention,144,23959651.0
1,Product Launch,123,20537548.0
2,Brand Awareness,115,19783668.0
3,E-commerce Revenue,112,17334820.0
4,Customer Acquisition,106,17505568.0


### Top KPI Results

In [168]:
query = """
MATCH (c:MarketingCampaign)-[r:KPI_RESULT]->(k:MarketingKPI)
RETURN k.name AS kpi_name,
       k.unit AS unit,
       count(r) AS measurement_count,
       avg(r.actual_value) AS avg_value,
       max(r.actual_value) AS max_value
ORDER BY measurement_count DESC
LIMIT 10;
"""

result = run_query(query)
print("Top 10 KPIs by measurement count:")
result

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: actual_value)} {position: line: 6, column: 14, offset: 163} for query: '\nMATCH (c:MarketingCampaign)-[r:KPI_RESULT]->(k:MarketingKPI)\nRETURN k.name AS kpi_name,\n       k.unit AS unit,\n       count(r) AS measurement_count,\n       avg(r.actual_value) AS avg_value,\n       max(r.actual_value) AS max_value\nORDER BY measurement_count DESC\nLIMIT 10;\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the dat

Top 10 KPIs by measurement count:


,kpi_name,unit,measurement_count,avg_value,max_value
0,Click Through Rate,Ratio,1793,None,None
1,Conversion Rate,Ratio,1793,None,None
2,Return On Ad Spend,Currency,1793,None,None
3,Impressions,Count,1793,None,None
4,Cost Per Acquisition,Currency,1793,None,None


## 4. Procurement Domain Queries

### Supplier Overview

In [169]:
query = """
MATCH (s:Supplier)
OPTIONAL MATCH (s)-[:HAS_CONTRACT]->(c:Contract)
OPTIONAL MATCH (s)-[:PLACED_ORDER]->(po:PurchaseOrder)-[:HAS_LINE]->(line:PurchaseOrderLine)
WITH s,
     count(DISTINCT c) AS contract_count,
     count(DISTINCT po) AS purchase_order_count,
     sum(line.quantity * line.unit_price) AS total_spend
RETURN s.name AS supplier_name,
       s.tier AS tier,
       s.country AS country,
       contract_count,
       purchase_order_count,
       coalesce(total_spend, 0) AS total_spend
ORDER BY total_spend DESC
LIMIT 15;
"""

result = run_query(query)
print("Top 15 suppliers by total spend:")
result

Top 15 suppliers by total spend:


,supplier_name,tier,country,contract_count,purchase_order_count,total_spend
0,Finch-Lewis,Preferred,RS,3,19,7.043305e+08
1,Ferguson Ltd,Preferred,LT,3,16,6.791845e+08
2,Marsh-Andrews,Preferred,MD,3,19,6.751910e+08
3,Griffiths Ltd,Preferred,PE,3,17,6.540004e+08
4,Lewis Ltd,Strategic,NL,3,17,6.500223e+08
5,Richardson Inc,Tactical,NI,3,17,6.493796e+08
6,Ward PLC,Strategic,RO,3,18,6.294299e+08
7,"Burton, Wheeler and Robinson",Preferred,KN,3,18,6.170056e+08
8,Hudson LLC,Tactical,AM,3,18,6.033241e+08
9,"Price, Talbot and Boyle",Approved,AZ,3,16,5.803835e+08


### High-Risk Suppliers

In [170]:
query = """
MATCH (s:Supplier)-[:HAS_RISK]->(r:SupplierRisk)
WHERE r.score >= 70
RETURN s.name AS supplier_name,
       s.tier AS tier,
       r.category AS risk_category,
       r.score AS risk_score,
       r.assessed_date AS assessed_date,
       r.notes AS notes
ORDER BY r.score DESC;
"""

result = run_query(query)
print("High-risk suppliers (risk score >= 70):")
result

High-risk suppliers (risk score >= 70):


,supplier_name,tier,risk_category,risk_score,assessed_date,notes
0,Lewis Ltd,Strategic,ESG,95.0,2025-08-23,Dolorem iusto saepe consequatur adipisci repud...
1,"Rees, Rose and Winter",Strategic,ESG,94.9,2025-08-22,Deleniti nulla facere vitae officia mollitia q...
2,Fry-Austin,Strategic,ESG,94.8,2025-06-23,Maiores quam labore ipsa nihil eligendi a labo...
3,"Martin, Skinner and Higgins",Strategic,Operational,94.7,2025-08-28,Consectetur explicabo corrupti consequatur.
4,Phillips-Lynch,Tactical,Operational,94.4,2025-06-18,Omnis aliquam laboriosam molestiae commodi eni...
...,...,...,...,...,...,...
269,"White, Wilkinson and Foster",Preferred,ESG,70.2,2025-09-27,Vel iste occaecati voluptas voluptas ullam quo...
270,Andrews-Ellis,Preferred,ESG,70.2,2025-06-22,Impedit nemo similique minima alias laudantium...
271,Richards-Smith,Preferred,Compliance,70.1,2025-07-28,Quo omnis nemo porro quaerat itaque.
272,"Rees, Rose and Winter",Strategic,GeoPolitical,70.1,2025-06-19,Praesentium quod enim vel officia unde laborum...


### Supplier Risk Summary by Category

In [171]:
query = """
MATCH (s:Supplier)-[:HAS_RISK]->(r:SupplierRisk)
RETURN r.category AS risk_category,
       count(DISTINCT s) AS supplier_count,
       avg(r.score) AS avg_score,
       max(r.score) AS max_score
ORDER BY avg_score DESC;
"""

result = run_query(query)
print("Supplier risk summary by category:")
result

Supplier risk summary by category:


,risk_category,supplier_count,avg_score,max_score
0,Compliance,102,70.693137,94.3
1,Operational,108,70.634259,94.7
2,Financial,108,70.484259,94.0
3,GeoPolitical,106,69.863208,93.9
4,ESG,110,69.310909,95.0


### Purchase Order Summary

In [172]:
query = """
MATCH (po:PurchaseOrder)-[:HAS_LINE]->(line:PurchaseOrderLine)
WITH po, sum(line.quantity * line.unit_price) AS po_total
RETURN po.status AS status,
       count(po) AS order_count,
       sum(po_total) AS total_amount,
       avg(po_total) AS avg_amount
ORDER BY order_count DESC;
"""

result = run_query(query)
print("Purchase orders by status:")
result

Purchase orders by status:


,status,order_count,total_amount,avg_amount
0,Delivered,542,6.063902e+09,1.118801e+07
1,Open,504,5.243447e+09,1.040367e+07
2,Closed,357,3.866004e+09,1.082914e+07
3,Approved,344,3.944631e+09,1.146695e+07


### Active Contracts Summary

In [173]:
query = """
MATCH (s:Supplier)-[:HAS_CONTRACT]->(c:Contract)
WHERE c.end_date IS NULL OR c.end_date >= date()
RETURN s.name AS supplier_name,
       c.contract_id AS contract_id,
       c.category AS category,
       c.value AS contract_value,
       c.currency AS currency,
       c.start_date AS start_date,
       c.end_date AS end_date
ORDER BY c.value DESC
LIMIT 20;
"""

result = run_query(query)
print("Top 20 active contracts by value:")
result

Top 20 active contracts by value:


,supplier_name,contract_id,category,contract_value,currency,start_date,end_date
0,Finch-Lewis,CON-00157,3D Printing,4977143.19,EUR,2023-01-01,None
1,"Wilkinson, Anderson and Williams",CON-00125,Software Licenses,4967667.84,EUR,2024-03-02,None
2,Thomas-White,CON-00030,Machining,4949521.40,EUR,2024-03-04,2027-02-11
3,"Allen, Saunders and Lee",CON-00189,3D Printing,4947526.83,EUR,2023-04-12,2026-03-15
4,Freeman PLC,CON-00300,Logistics,4946036.53,EUR,2024-08-02,None
5,"Burton, Wheeler and Robinson",CON-00258,Consulting,4936271.35,EUR,2023-01-24,None
6,"Clarke, Moran and Smith",CON-00165,Injection Molding,4924239.88,EUR,2022-11-18,2026-10-31
7,Lewis Ltd,CON-00062,Consulting,4916626.76,EUR,2023-10-04,2026-10-23
8,Freeman PLC,CON-00301,Injection Molding,4911961.43,EUR,2025-02-10,2026-08-19
9,Smith-Taylor,CON-00267,Facility Services,4878225.73,EUR,2025-03-19,None


### Purchase Order Lines by Product Category

In [174]:
query = """
MATCH (line:PurchaseOrderLine)
RETURN line.product_category AS product_category,
       count(line) AS line_count,
       sum(line.quantity) AS total_quantity,
       sum(line.quantity * line.unit_price) AS total_value,
       avg(line.unit_price) AS avg_unit_price
ORDER BY total_value DESC
LIMIT 15;
"""

result = run_query(query)
print("Top 15 product categories by spend:")
result

Top 15 product categories by spend:


,product_category,line_count,total_quantity,total_value,avg_unit_price
0,Packaging,584,1441534.84,2.502593e+09,1731.670257
1,3D Printing,545,1401672.02,2.445656e+09,1760.898716
2,Facility Services,473,1214573.88,2.122288e+09,1735.319493
3,Consulting,461,1213843.12,2.013573e+09,1664.343970
4,PCB Assembly,443,1079310.66,1.778469e+09,1683.629368
5,Logistics,355,945385.43,1.758252e+09,1846.405915
6,Machining,378,953793.98,1.680831e+09,1734.908492
7,Software Licenses,367,947387.68,1.651788e+09,1721.707275
8,Injection Molding,372,953981.65,1.599017e+09,1695.850941
9,Steel Raw Materials,353,923345.87,1.565517e+09,1736.846261


## 5. Cross-Domain Analytics

### Department-Brand Connections

In [175]:
# Find connections between departments and brands through employees and campaigns
query = """
MATCH (d:Department)<-[:WORKS_IN]-(e:Employee)
WHERE d.name IN ['Marketing', 'HR', 'Procurement']
WITH d, count(e) AS employee_count
OPTIONAL MATCH (d)<-[:WORKS_IN]-(e2:Employee)-[*1..2]-(b:Brand)
RETURN d.name AS department,
       employee_count,
       count(DISTINCT b) AS connected_brands
ORDER BY connected_brands DESC;
"""

result = run_query(query)
print("Department connections to brands:")
result

Department connections to brands:


,department,employee_count,connected_brands
0,Marketing,30,0
1,HR,32,0


### Supplier Contract Product Categories

In [176]:
# Check product categories across suppliers
query = """
MATCH (s:Supplier)-[:PLACED_ORDER]->(po:PurchaseOrder)-[:HAS_LINE]->(line:PurchaseOrderLine)
WITH s, collect(DISTINCT line.product_category) AS categories, count(DISTINCT po) AS order_count
RETURN s.name AS supplier_name,
       s.tier AS tier,
       size(categories) AS category_count,
       categories,
       order_count
ORDER BY category_count DESC
LIMIT 10;
"""

result = run_query(query)
print("Top suppliers by product category diversity:")
result

Top suppliers by product category diversity:


,supplier_name,tier,category_count,categories,order_count
0,Spencer and Sons,Strategic,3,"[Logistics, Steel Raw Materials, Packaging]",5
1,"Jones, Khan and Parker",Strategic,3,"[PCB Assembly, Consulting, Packaging]",12
2,"Short, Perkins and Smith",Strategic,3,"[Facility Services, Machining, Consulting]",11
3,"Turnbull, Greenwood and Atkinson",Strategic,3,"[Steel Raw Materials, Machining, Facility Serv...",7
4,Foster Inc,Strategic,3,"[Logistics, Software Licenses, Steel Raw Mater...",9
5,Ward PLC,Strategic,3,"[Software Licenses, 3D Printing, Injection Mol...",18
6,Phillips-Lynch,Tactical,3,"[Packaging, Software Licenses, Facility Services]",7
7,"King, Howarth and Bruce",Tactical,3,"[Injection Molding, Steel Raw Materials, Facil...",15
8,"Webb, Gray and Patel",Approved,3,"[Consulting, Packaging, Software Licenses]",15
9,Taylor-White,Approved,3,"[PCB Assembly, Injection Molding, Software Lic...",18


## 6. 3D Graph Visualization

### Export Graph Data for 3D Visualization

In [177]:
# Export nodes and links for 3D visualization
import json
from datetime import datetime, date

def serialize_neo4j_types(obj):
    """Convert Neo4j types to JSON-serializable formats"""
    if hasattr(obj, 'isoformat'):  # datetime, date, time
        return obj.isoformat()
    elif isinstance(obj, dict):
        return {k: serialize_neo4j_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [serialize_neo4j_types(item) for item in obj]
    else:
        return obj

query = """
MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
WITH
  collect(DISTINCT { 
    id: elementId(n), 
    label: head(labels(n)), 
    name: coalesce(n.name, n.first_name, n.employee_id, n.campaign_id, n.supplier_id, ''),
    props: properties(n) 
  }) AS nodes,
  collect(DISTINCT { 
    source: elementId(n), 
    target: elementId(m), 
    type: type(r), 
    props: properties(r) 
  }) AS links
RETURN { nodes: nodes, links: links } AS graph;
"""

print("Fetching graph data from Neo4j...")
with driver.session() as session:
    result = session.run(query)
    graph_data = result.single()['graph']

# Remove null links (from OPTIONAL MATCH where no relationship exists)
graph_data['links'] = [link for link in graph_data['links'] if link['target'] is not None]

# Serialize Neo4j types to JSON-compatible format
graph_data = serialize_neo4j_types(graph_data)

print(f"✓ Exported {len(graph_data['nodes']):,} nodes")
print(f"✓ Exported {len(graph_data['links']):,} relationships")

# Save to JSON file
output_path = os.path.join(project_root, 'notebooks', 'graph.json')
with open(output_path, 'w') as f:
    json.dump(graph_data, f, indent=2)

print(f"✓ Saved to {output_path}")

Fetching graph data from Neo4j...
✓ Exported 9,793 nodes
✓ Exported 27,131 relationships
✓ Saved to /Users/ivan/FSFM/01_Courses/Coop/Helixgraph/HEL-20/HelixGraph/notebooks/graph.json


### Generate 3D Visualization HTML

In [178]:
# Generate HTML file for 3D visualization with embedded data
import json

# Read the graph data
graph_json_path = os.path.join(project_root, 'notebooks', 'graph.json')
with open(graph_json_path, 'r') as f:
    graph_data_embedded = json.load(f)

# Convert to JavaScript variable
graph_data_js = json.dumps(graph_data_embedded)

html_content = f'''<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>HelixGraph 3D Visualization - All Departments</title>
  <style>
    html, body {{ height: 100%; margin: 0; font-family: Arial, sans-serif; }}
    #3d-graph {{ width: 100%; height: 100vh; display:block; }}
    .info {{ 
      position: absolute; 
      left: 10px; 
      top: 10px; 
      color: #fff; 
      z-index:100; 
      background: rgba(0,0,0,0.7); 
      padding:10px 15px; 
      border-radius:8px; 
      font-size: 14px;
      line-height: 1.6;
    }}
    .stats {{
      position: absolute;
      right: 10px;
      top: 10px;
      color: #fff;
      z-index: 100;
      background: rgba(0,0,0,0.7);
      padding: 10px 15px;
      border-radius: 8px;
      font-size: 12px;
    }}
    .legend {{
      position: absolute;
      left: 10px;
      bottom: 10px;
      color: #fff;
      z-index: 100;
      background: rgba(0,0,0,0.7);
      padding: 10px 15px;
      border-radius: 8px;
      font-size: 11px;
      max-height: 300px;
      overflow-y: auto;
    }}
    .legend-item {{
      display: flex;
      align-items: center;
      margin: 4px 0;
    }}
    .legend-color {{
      width: 12px;
      height: 12px;
      border-radius: 50%;
      margin-right: 8px;
    }}
  </style>
</head>
<body>
  <div id="3d-graph"></div>
  <div class="info">
    <strong>HelixGraph 3D Visualization</strong><br>
    🖱️ Drag to rotate<br>
    📜 Scroll to zoom<br>
    🎯 Click node to center
  </div>
  <div class="stats" id="stats">Loading...</div>
  <div class="legend" id="legend"></div>

  <!-- 3D Force Graph (from unpkg CDN) -->
  <script src="https://unpkg.com/three@0.152.2/build/three.min.js"></script>
  <script src="https://unpkg.com/3d-force-graph"></script>

  <script>
    // Embedded graph data (no external file needed - CORS safe)
    const data = {graph_data_js};
    
    console.log('Loaded embedded graph data');
    console.log('Nodes:', data.nodes.length, 'Links:', data.links.length);
    
    // Calculate statistics
    const nodeCounts = {{}};
    data.nodes.forEach(node => {{
      nodeCounts[node.label] = (nodeCounts[node.label] || 0) + 1;
    }});
    
    const linkCounts = {{}};
    data.links.forEach(link => {{
      linkCounts[link.type] = (linkCounts[link.type] || 0) + 1;
    }});
    
    // Display statistics
    const statsDiv = document.getElementById('stats');
    statsDiv.innerHTML = '<strong>Graph Statistics</strong><br>' +
      `Nodes: ${{data.nodes.length.toLocaleString()}}<br>` +
      `Links: ${{data.links.length.toLocaleString()}}`;
    
    // Create legend
    const legendDiv = document.getElementById('legend');
    legendDiv.innerHTML = '<strong>Node Types</strong><br>';
    
    // Initialize graph
    const Graph = ForceGraph3D()
      (document.getElementById('3d-graph'))
      .graphData(data)
      .nodeAutoColorBy('label')
      .nodeLabel(node => {{
        const name = node.name || node.id;
        return `${{node.label}}: ${{name}}`;
      }})
      .linkDirectionalParticles(1)
      .linkDirectionalParticleSpeed(0.008)
      .linkWidth(link => 0.5)
      .linkOpacity(0.3)
      .nodeThreeObject(node => {{
        // Different sizes for different node types
        let size = 2;
        if (node.label === 'Department') size = 8;
        else if (node.label === 'Employee') size = 3;
        else if (node.label === 'Supplier') size = 5;
        else if (node.label === 'MarketingCampaign') size = 4;
        else if (node.label === 'Brand') size = 6;
        
        const obj = new THREE.Mesh(
          new THREE.SphereGeometry(size),
          new THREE.MeshLambertMaterial({{ 
            color: node.__color,
            transparent: true,
            opacity: 0.85
          }})
        );
        return obj;
      }})
      .onNodeClick(node => {{
        // Center camera on node
        const distance = 120;
        const distRatio = 1 + distance / Math.hypot(node.x, node.y, node.z);
        Graph.cameraPosition(
          {{ 
            x: node.x * distRatio, 
            y: node.y * distRatio, 
            z: node.z * distRatio 
          }},
          node,
          1500
        );
        console.log('Clicked node:', node);
      }})
      .onNodeHover(node => {{
        document.body.style.cursor = node ? 'pointer' : 'default';
      }});
    
    // Populate legend with node colors
    setTimeout(() => {{
      const colorMap = {{}};
      data.nodes.forEach(node => {{
        if (!colorMap[node.label]) {{
          colorMap[node.label] = node.__color;
        }}
      }});
      
      Object.entries(nodeCounts).sort((a, b) => b[1] - a[1]).forEach(([label, count]) => {{
        const color = colorMap[label] || '#999';
        legendDiv.innerHTML += `
          <div class="legend-item">
            <div class="legend-color" style="background-color: ${{color}}"></div>
            <span>${{label}} (${{count.toLocaleString()}})</span>
          </div>
        `;
      }});
    }}, 500);
    
    // Initial zoom to fit
    Graph.zoomToFit(800);
  </script>
</body>
</html>
'''

# Save HTML file
html_path = os.path.join(project_root, 'notebooks', 'graph_3d.html')
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✓ Generated 3D visualization HTML with embedded data")
print(f"✓ Saved to {html_path}")
print(f"✓ File size: {len(html_content):,} bytes")
print("\n📊 To view the 3D graph:")
print(f"   Open {html_path} in your browser")
print("   (No CORS issues - data is embedded in HTML)")

✓ Generated 3D visualization HTML with embedded data
✓ Saved to /Users/ivan/FSFM/01_Courses/Coop/Helixgraph/HEL-20/HelixGraph/notebooks/graph_3d.html
✓ File size: 10,043,428 bytes

📊 To view the 3D graph:
   Open /Users/ivan/FSFM/01_Courses/Coop/Helixgraph/HEL-20/HelixGraph/notebooks/graph_3d.html in your browser
   (No CORS issues - data is embedded in HTML)


### View in Jupyter (Inline)

You can also view the 3D graph inline in Jupyter using an IFrame:

In [179]:
from IPython.display import IFrame

# Display the 3D visualization inline
IFrame(src='graph_3d.html', width=1000, height=700)

## Cleanup

In [180]:
# Close the driver connection
# Note: Commented out to allow running cells multiple times in the notebook
# Uncomment if this is your final operation
# driver.close()
# print("✓ Connection closed")

print("ℹ️  Driver still active (allows re-running cells)")
print("   Run driver.close() manually when completely done")

ℹ️  Driver still active (allows re-running cells)
   Run driver.close() manually when completely done
